In [1]:
import numpy as np
import theano
import lasagne
import random
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet
from sklearn.metrics import classification_report, confusion_matrix
import IOutils

In [2]:
ds = IOutils.data_streamer2()
vt = IOutils.VectorTransformer()

In [3]:
X_val, Y_val = ds.next()

In [9]:
Y_val = vt.transform(Y_val)

In [12]:
non_zero_data = np.where(Y_val)
X_val, Y_val = X_val[non_zero_data,:],Y_val[non_zero_data]

In [13]:
X_val = X_val/X_val.max()

In [14]:
nn = NeuralNet(layers=[
        ('input', layers.InputLayer),
        ('hidden', layers.DenseLayer),
        ('hidden2', layers.DenseLayer),
        ('output', layers.DenseLayer)
    ],
    input_shape=(None, 42),
    hidden_num_units= 100,
    hidden2_num_units=100,
    output_num_units = len(vt.unique_rows),
    output_nonlinearity=lasagne.nonlinearities.softmax,
    update_learning_rate = 0.1,
    update = nesterov_momentum,
    max_epochs=100,
    verbose=10000)

In [21]:
X_next, Y_next = ds.next()
X_next = X_next/X_next.max()
X_next[-256:, 32:]=0
zipped = zip(X_next,Y_next)
random.shuffle(zipped)
X,Y = zip(*zipped)

X_next = np.array(X)
Y_next = np.array(Y)
X_next = X_next.astype(np.float)
Y_next = vt.transform(Y_next)

non_zero_labels = np.where(Y_next > 0)
X_next, Y_next = X_next[non_zero_labels, :], Y_next[non_zero_labels]
X_next = X_next.reshape(-1, 42)
nn.fit(X_next, Y_next)

# Neural Network with 15612 learnable parameters

## Layer information

  #  name       size
---  -------  ------
  0  input        42
  1  hidden      100
  2  hidden2     100
  3  output       12

  epoch    train loss    valid loss    train/val    valid acc  dur
-------  ------------  ------------  -----------  -----------  -----
      1       1.92955       1.67483      1.15209      0.41857  0.19s
      2       1.46964       1.33628      1.09980      0.51768  0.16s
      3       1.29620       1.23336      1.05095      0.55674  0.16s
      4       1.21334       1.16341      1.04292      0.58549  0.16s
      5       1.14766       1.11132      1.03270      0.59962  0.16s
      6       1.09067       1.03598      1.05280      0.61853  0.15s
      7       1.04318       0.98832      1.05551      0.63569  0.15s
      8       0.99637       0.92681      1.07505      0.65235  0.15s
      9       0.95166       0.88332      1.07736      0.65564  0.15s
     10       0.91522       0.84502      1.0

NeuralNet(X_tensor_type=None,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x1073b4c10>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x1073b4b90>,
     custom_score=None, hidden2_num_units=100, hidden_num_units=100,
     input_shape=(None, 42),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('hidden', <class 'lasagne.layers.dense.DenseLayer'>), ('hidden2', <class 'lasagne.layers.dense.DenseLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=None, max_epochs=100, more_params={},
     objective=<function objective at 0x1073bf6e0>,
     objective_loss_function=<function categorical_crossentropy at 0x1071612a8>,
     on_batch_finished=[],
     on_epoch_finished=[<nolearn.lasagne.handlers.PrintLog instance at 0x112444f38>],
     on_training_finished=[],
     on_training_started=[<nolearn.lasagne.handlers.PrintLayerInfo instance at 0x112444f80>],
     output_nonlinearity=<function softmax at 0x1

In [25]:
predicted = nn.predict(X_val.reshape(-1,42).astype(np.float))

In [26]:
confusion_matrix(Y_val, predicted)

array([[ 249,  108, 2267,    0,    2,   64,   70,   50,    0,    0],
       [  90,   73, 1120,    0,    0,   48,    8,  104,    2,    1],
       [ 116,   48, 4660,    0,    0,   89,    5,  149,    0,   33],
       [  23,    6,  261,    0,    0,    0,   52,    1,    0,    0],
       [  64,   19,  658,    0,    1,   26,   50,   26,    0,    0],
       [ 217,  132,  764,   11,    0,   17,    0,  182,    0,   29],
       [  17,   52,  384,    0,    0,   25,    3,   18,    1,    1],
       [ 350,  122, 2464,    4,    0,   77,   75,   61,    0,    0],
       [ 352,  547, 2447,   10,    0,  132,   43,   47,   72,   98],
       [ 526,  289, 2518,    3,    0,  154,   13,   77,   37,  131]])

In [27]:
print classification_report(Y_val, predicted)

             precision    recall  f1-score   support

          1       0.12      0.09      0.10      2810
          2       0.05      0.05      0.05      1446
          3       0.27      0.91      0.41      5100
          4       0.00      0.00      0.00       343
          5       0.33      0.00      0.00       844
          6       0.03      0.01      0.02      1352
          7       0.01      0.01      0.01       501
          8       0.09      0.02      0.03      3153
         10       0.64      0.02      0.04      3748
         11       0.45      0.03      0.06      3748

avg / total       0.28      0.23      0.13     23045

